In [3]:
import magma as m
from mantle import DefineRegister

To implement a counter we can use the `DefineRegister` primitive and the `+` operator for `m.UInt` instances.

In [7]:
Reg4 = DefineRegister(4, _type=m.UInt)

class Counter4(m.Circuit):
    name = "Counter4"
    IO = ["count", m.Out(m.UInt(4))] + m.ClockInterface()
    @classmethod
    def definition(io):
        reg4 = Reg4()
        count = reg4.O + m.uint(1,4)
        m.wire(count, reg4.I)
        m.wire(reg4.O, io.count)
        m.wireclock(io, reg4)

We use the Python simulator to generate a waveform of our circuit.
* `step` steps the clock
* `evaluate` evaluates the circuit
* `get_value` returns the value of the circuit ports

In [9]:
from magma.simulator.python_simulator import PythonSimulator
from magma.scope import Scope

simulator = PythonSimulator(Counter4)
scope = Scope()
waveforms = []
for i in range(18):
    for j in range(2):
        simulator.step()
        simulator.evaluate()
        clk = simulator.get_value(Counter4.CLK, scope)
        O = simulator.get_value(Counter4.count, scope)
        waveforms.append(O + [clk])

AttributeError: 'PythonSimulator' object has no attribute 'step'

magma provides a helper function to plot waveform diagrams

In [ ]:
from magma.waveform import waveform

waveform(waveforms, ["O[0]", "O[1]", "O[2]", "O[3]", "CLK"])

We can also inspect the generated verilog for our counter.

In [ ]:
m.compile("build/counter", Counter4, include_coreir=True)

In [ ]:
with open("build/counter.v", "r") as counter_verilog:
    print(counter_verilog.read())